# Lab 9-1: MNIST Softmax&nn
이 파일은 모두딥 시즌2의 9-1,2 내용에 해당합니다

## Imports

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

device = 'cuda' if torch.cuda.is_available() else 'cpu'

#for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

## Data

In [2]:
#parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [3]:
#MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform= transforms.ToTensor(),
                          download= True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform= transforms.ToTensor(),
                          download= True)

#datalodaer
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



## Softmax

### Model (Softmax)

In [4]:
linear = torch.nn.Linear(28*28, 10, bias = True).to(device)

#initialization
torch.nn.init.normal_(linear.weight)

#cost & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(linear.parameters(), lr= learning_rate)

### Training & Testing (Softmax)

In [5]:
total_batch = len(data_loader)

for epoch in range(training_epochs):
  avg_cost = 0

  for X, Y in data_loader:
    # reshape input image into [batch_size by 784]
    # label is not one-hot encoded
    X = X.view(-1, 28*28).to(device)
    Y = Y.to(device)

    hypothesis = linear(X)
    cost = criterion(hypothesis, Y)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch
  
  print('Epoch: {:4d} Cost: {:.9f}'.format(epoch+1, avg_cost))
print('Learning Finished.')

Epoch:    1 Cost: 5.645695210
Epoch:    2 Cost: 1.696827292
Epoch:    3 Cost: 1.121626616
Epoch:    4 Cost: 0.882780135
Epoch:    5 Cost: 0.748860061
Epoch:    6 Cost: 0.661943853
Epoch:    7 Cost: 0.599809587
Epoch:    8 Cost: 0.553705037
Epoch:    9 Cost: 0.517529070
Epoch:   10 Cost: 0.488772631
Epoch:   11 Cost: 0.465197325
Epoch:   12 Cost: 0.445370257
Epoch:   13 Cost: 0.428630620
Epoch:   14 Cost: 0.413612187
Epoch:   15 Cost: 0.401216000
Learning Finished.


In [6]:
#test
with torch.no_grad():
  X_test = mnist_test.data.view(-1, 28*28).float().to(device)
  Y_test = mnist_test.targets.to(device)

  prediction = linear(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())

  #Get one and predict
  r = random.randint(0, len(mnist_test) - 1)
  X_single = mnist_test.data[r:r+1].view(-1, 28*28).float().to(device)
  Y_single = mnist_test.targets[r:r+1].to(device)

  print('Label:', Y_single.item())
  single_prediction = linear(X_single)
  print('Prediction:', torch.argmax(single_prediction, 1).item())

Accuracy: 0.8875166773796082
Label: 9
Prediction: 9


## nn

### Model (nn)

In [7]:
#nn layers
linear1 = torch.nn.Linear(28*28, 256, bias= True)
linear2 = torch.nn.Linear(256, 256, bias= True)
linear3 = torch.nn.Linear(256, 10, bias= True)

relu = torch.nn.ReLU()

#initialization
torch.nn.init.normal_(linear1.weight)
torch.nn.init.normal_(linear2.weight)
torch.nn.init.normal_(linear3.weight)

#model
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device) #linear3이후에는 softmax가 적용되므로 relu가 x

#cost & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

### Training & Testing (nn)

In [8]:
total_batch = len(data_loader)

for epoch in range(training_epochs):
  avg_cost = 0

  for X, Y in data_loader:
    # reshape input image into [batch_size by 784]
    # label is not one-hot encoded
    X = X.view(-1, 28*28).to(device)
    Y = Y.to(device)

    hypothesis = model(X)
    cost = criterion(hypothesis, Y)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch
  
  print('Epoch: {:4d} Cost: {:.9f}'.format(epoch+1, avg_cost))
print('Learning Finished.')

Epoch:    1 Cost: 130.755035400
Epoch:    2 Cost: 35.983997345
Epoch:    3 Cost: 22.489233017
Epoch:    4 Cost: 15.643187523
Epoch:    5 Cost: 11.169998169
Epoch:    6 Cost: 8.092410088
Epoch:    7 Cost: 6.038753986
Epoch:    8 Cost: 4.475240231
Epoch:    9 Cost: 3.309149981
Epoch:   10 Cost: 2.393388033
Epoch:   11 Cost: 1.835352182
Epoch:   12 Cost: 1.354556918
Epoch:   13 Cost: 1.071245193
Epoch:   14 Cost: 0.866881013
Epoch:   15 Cost: 0.652935147
Learning Finished.


In [9]:
#test
with torch.no_grad():
  X_test = mnist_test.data.view(-1, 28*28).float().to(device)
  Y_test = mnist_test.targets.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())

  #Get one and predict
  r = random.randint(0, len(mnist_test) - 1)
  X_single = mnist_test.data[r:r+1].view(-1, 28*28).float().to(device)
  Y_single = mnist_test.targets[r:r+1].to(device)

  print('Label:', Y_single.item())
  single_prediction = linear(X_single)
  print('Prediction:', torch.argmax(single_prediction, 1).item())

Accuracy: 0.991349995136261
Label: 4
Prediction: 4
